In [2]:
import cv2
import numpy as np
import os

from keras.src.callbacks import CSVLogger
from keras.src.layers import Masking
from matplotlib import pyplot as plt
import time
import mediapipe as mp

2024-06-06 13:54:26.138033: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 13:54:27.147514: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
sequence_length = 30  # Number of frames per sequence
shape_no_face = (sequence_length, 258)
num_classes = 4
actions = ['hallo','du','idle','a']  # List all your actions including idle
DATA_PATH = 'CollectionBase/MP_Data/30frame_no_face_+idle'  # Update with your data path
landmarks = [False,True,True,True] # Select True/False for different landmarks (face, pose, left hand, right hand)

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def extract_keypoints(results):
    arr = []
    if(landmarks[0]):
        face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
        arr.append(face)
    if(landmarks[1]):
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
        arr.append(pose)
    if(landmarks[2]):
        lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
        arr.append(lh)
    if(landmarks[3]):
        rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
        arr.append(rh)
    return np.concatenate(arr)

def draw_landmarks(image, results):
    if(landmarks[0]):
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    if(landmarks[1]):
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    if(landmarks[2]):
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    if(landmarks[3]):
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)


def draw_styled_landmarks(image, results):
    if(landmarks[0]):
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                  mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                  )
    if(landmarks[1]):
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                  )
    if(landmarks[2]):
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                  )
    if(landmarks[3]):
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

## Labeling

In [4]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Paths and parameters

# Initialize sequences and labels
sequences, labels = [], []

# Create a mapping of labels to numbers
label_map = {label: num for num, label in enumerate(actions)}

# Loop through each action and each sequence within that action
for action in actions:
    action_dir = os.path.join(DATA_PATH, action)
    for sequence in np.array(os.listdir(action_dir)).astype(int):
        window = []
        for frame_num in range(sequence_length):
            frame_path = os.path.join(action_dir, str(sequence), f"{frame_num}.npy")
            res = np.load(frame_path)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

# Convert sequences and labels to numpy arrays
X = np.array(sequences)
y = to_categorical(labels).astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# Print the shape of the data
print(f"Total sequences: {len(sequences)}")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


Total sequences: 120
X shape: (120, 30, 258)
y shape: (120, 4)
X_train shape: (114, 30, 258), y_train shape: (114, 4)
X_test shape: (6, 30, 258), y_test shape: (6, 4)


# Version 0.1.0


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
import os

# Define the L2 regularization factor
l2_factor = 0.01

model = Sequential()

# Input layer
model.add(Input(shape=shape_no_face))

# Adding LSTM layers with L2 regularization
model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(LSTM(128, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

# Fully connected layer with L2 regularization
model.add(Dense(256, activation='relu', kernel_regularizer=l2(l2_factor)))
model.add(Dropout(0.3))

# Output layer for num_classes with L2 regularization
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=l2(l2_factor)))

# Set up CSVLogger
csv_logger = CSVLogger('training_log.csv', append=True)

# Set up TensorBoard
log_dir = os.path.join("logs", "fit", "sign_language_recognition")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# List of callbacks
callbacks = [csv_logger, tensorboard_callback, early_stopping]

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=callbacks)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Dropout, Masking

model = Sequential()
model.add(Masking(mask_value=0., input_shape=(30, 258)))  # Adjust input shape accordingly
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))  # num_classes is the number of sign language classes

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


# Most accuracte

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
import os

# Define the L2 regularization factor
l2_factor = 0.01

model = Sequential()

# Input layer
model.add(Masking(mask_value=0., input_shape=shape_no_face))  # Adjust input shape accordingly

# Adding LSTM layers with L2 regularization
model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(LSTM(128, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

# Fully connected layer with L2 regularization
model.add(Dense(256, activation='relu', kernel_regularizer=l2(l2_factor)))
model.add(Dropout(0.3))

# Output layer for num_classes with L2 regularization
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=l2(l2_factor)))

# Set up CSVLogger
csv_logger = CSVLogger('training_log.csv', append=True)

# Set up TensorBoard
log_dir = os.path.join("logs", "fit", "sign_language_recognition")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# List of callbacks
callbacks = [csv_logger, tensorboard_callback, early_stopping]

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=callbacks)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

# Experimental

# Testing

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
import os

In [12]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

model = tf.keras.models.load_model('ai.h5')

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Mirror the frame
        frame = cv2.flip(frame, 1)

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        cap.release()
        cv2.destroyAllWindows()
cap.release()
cv2.destroyAllWindows()

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    # Ensure that the size of 'res' and 'colors' is the same
    if len(res) != len(colors):
        print("Size of 'res' and 'colors' does not match. Please check your inputs.")
        return output_frame
    for num, prob in enumerate(res):
        max_prob = np.max(prob)  # get the maximum probability
        cv2.rectangle(output_frame, (0,60+num*40), (int(max_prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_frame
colors = [(245,117,16), (117,245,16), (80,22,10), (121,22,76)]
cap = cv2.VideoCapture(0)  # Start the webcam
ret, image = cap.read()  # Read a frame from the webcam
cap.release()  # Close the webcam

#plt.figure(figsize=(18,18))
#plt.imshow(prob_viz(res, actions, image, colors))
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    collecting = False
    frame_count = 0
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Check if 'g' is pressed
        if cv2.waitKey(10) & 0xFF == ord('g'):
            collecting = True
            sequence = []  # Reset the sequence
            frame_count = 0  # Reset the frame count

        # 2. Prediction logic
        if collecting and frame_count < 30:
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]
            frame_count += 1

            if frame_count == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                collecting = False  # Stop collecting after 30 frames

                #3. Viz logic
                if np.unique(predictions[-10:])[0]==np.argmax(res):
                    if res[np.argmax(res)] > threshold:

                        if len(sentence) > 0:
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5:
                    sentence = sentence[-5:]

                # Viz probabilities
                image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord(
                'q'):
            break
    cap.release()
    cv2.destroyAllWindows()

2024-06-06 13:59:57.498651: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-06 13:59:57.499415: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
I0000 00:00:1717675197.796688    6097 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1717675197.798447    6717 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.3-1pop1~1711635559~22.04~7a9f319), renderer: AMD Radeon Graphics

<class 'mediapipe.python.solution_base.SolutionOutputs'>


I0000 00:00:1717675198.444150    6097 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1717675198.446224    6739 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.3-1pop1~1711635559~22.04~7a9f319), renderer: AMD Radeon Graphics (radeonsi, renoir, LLVM 15.0.7, DRM 3.57, 6.8.0-76060800daily20240311-generic)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
du
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
du
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
du
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
du
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
hallo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
du
